In [6]:
import cv2
import imutils
import numpy as np
import pytesseract
import requests
import json
import xmltodict
import re

In [32]:
vimal_model = cv2.face.LBPHFaceRecognizer_create()
vimal_model.read('lbph_car_detection.yml')
cap = cv2.VideoCapture("../cars1.mp4")
fcount=0
while True:
    ret, frames = cap.read()
    gray = cv2.cvtColor(frames, cv2.COLOR_BGR2GRAY)
    results = vimal_model.predict(gray)
    if results[1] <500:
        confidence = int( 100 * (1 - (results[1])/400) )
    if confidence>= 85:
        fcount =fcount+1
    if fcount == 45:
        cv2.imwrite("photo.jpeg",frames)
        print("Car Detected")
        break
    else:
        print("NO car detected")
cv2.imshow('photo.jpeg',frames)
cv2.waitKey()=='13'
cv2.destroyAllWindows()
cap.release()

NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
NO car detected
Car Detected


In [33]:
pytesseract.pytesseract.tesseract_cmd = r'D:\pytesseract\tesseract.exe'
img = cv2.imread('photo.jpeg',cv2.IMREAD_COLOR)
img = cv2.resize(img, (500,600) )
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
gray = cv2.bilateralFilter(gray, 13, 15, 15) 
edged = cv2.Canny(gray, 30, 200) 
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
    detected = 1
    
if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)

(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

text = pytesseract.image_to_string(Cropped, config='--psm 11')
print("License Plate Recognition\n")
print("Detected license plate Number is:",text)
img = cv2.resize(img,(500,800))
Cropped = cv2.resize(Cropped,(400,200))
cv2.imshow('car',img)
cv2.waitKey()=='13'
cv2.destroyAllWindows()
cv2.imshow('Cropped',Cropped)
cv2.waitKey()=='13'
cv2.destroyAllWindows()
text = re.sub('[^A-Za-z0-9]+', '', text)
final_text = text[-10:]
print(final_text)

License Plate Recognition

Detected license plate Number is: =UP 15BB1516

UP15BB1516


In [34]:
vehicle_reg_no = final_text.strip() #NumberPlate deteted
username = "meem9" #API_user name
url = "http://www.regcheck.org.uk/api/reg.asmx/CheckIndia?RegistrationNumber=" + vehicle_reg_no + "&username="+username
url=url.replace(" ","%20")
r = requests.get(url)
n = xmltodict.parse(r.content)
k = json.dumps(n)
df = json.loads(k)
l=df["Vehicle"]["vehicleJson"]
p=json.loads(l)
s="Your car's details are:\n"+"Owner name: "+str(p['Owner'])+"\n"+"Car Company: "+str(p['CarMake']['CurrentTextValue'])+"\n"+"Car Model: "+str(p['CarModel']['CurrentTextValue'])+"\n"+"Fuel Type: "+str(p['FuelType']['CurrentTextValue'])+"\n"+"Registration Year: "+str(p['RegistrationYear'])+"\n"+"Insurance: "+str(p['Insurance'])+"\n"+"Vehicle ID: "+str(p['VechileIdentificationNumber'])+"\n"+"Engine No.: "+str(p['EngineNumber'])+"\n"+"Location RTO: "+str(p['Location'])
print(s)

Your car's details are:
Owner name: ABHINAY KUMAR
Car Company: HYUNDAI
Car Model: I20 SPORTZ 1.4 DSL BSIV
Fuel Type: DIESEL
Registration Year: 2012
Insurance: 2021-10-16
Vehicle ID: MALBB51RLCM476135J
Engine No.: D4FCCU191935
Location RTO: RTO, MEERUT
